In [5]:
from typing import Final

from bayes_opt import BayesianOptimization
from bayes_opt.event import Events
from bayes_opt.logger import JSONLogger
from bayes_opt.util import load_logs
from numpy import where
from pandas import DataFrame, Series, concat, read_csv
from sklearn.metrics import f1_score
from sklearn.svm import OneClassSVM

MIN_SAMPLES: Final[int] = 0
LOGS_PATH: Final[str] = "../reports/logs.log"
logger = JSONLogger(path=LOGS_PATH)

In [6]:
X_train = read_csv("../data/PAMAP2/x_train_data.csv")
X_test = read_csv("../data/PAMAP2/x_test_data.csv")
y_train = read_csv("../data/PAMAP2/y_train_data.csv")
y_test = read_csv("../data/PAMAP2/y_test_data.csv")

X_train["activity"] = y_train  # First 80% of the data
X_test["activity"] = y_test  # Last 20% of the data

MIN_SAMPLES = X_train["activity"].value_counts().min()

models: dict[int, OneClassSVM] = {}
training_data: DataFrame
testing_data: DataFrame
train_targets: Series
test_targets: Series

In [7]:
def objective_function(nu: float, gamma: float) -> float:
    """
    Objective function to optimize F1-Score on the test set.

    Args:
        nu (float): nu param to evaluate.
        gamma (float): gamma param to evaluate.

    Returns:
        float: F1-Score on the test set of this iteration.
    """
    oc_svm = OneClassSVM(kernel="rbf", nu=nu, gamma=gamma).fit(training_data)

    f1_train = f1_score(
        train_targets,
        where(oc_svm.predict(training_data) == 1, False, True),
        average="macro",
    )
    f1_test = f1_score(
        test_targets,
        where(oc_svm.predict(testing_data) == 1, False, True),
        average="macro",
    )
    print(f"\nF1 Score (Train): {f1_train}\nF1 Score (Test): {f1_test}")
    return float(f1_test)

In [8]:
novelty: DataFrame = X_train[X_train["activity"] == 1].head(MIN_SAMPLES)
current_data: DataFrame = X_train[~X_train.index.isin(novelty.index)].head(MIN_SAMPLES)
current_data.loc[:, "isNovelty"], novelty.loc[:, "isNovelty"] = False, True

sampled_data = current_data.sample(n=int(0.2 * len(current_data)), random_state=42)
novelty = concat([novelty, sampled_data])

training_data = current_data.drop(columns=["isNovelty"])
train_targets = current_data["isNovelty"]
# only current activity (as novelty)
testing_data = novelty.drop(columns=["isNovelty"])
test_targets = novelty["isNovelty"]

oc_svm, _ = objective_function(0.25, 0.005)


F1 Score (Train): 0.4282665791201576
F1 Score (Test): 0.843613888836446


In [22]:
for activity in X_train["activity"].unique():
    print(f"Activity: {activity}")
    novelty: DataFrame = X_train[X_train["activity"] == activity].head(MIN_SAMPLES)
    current_data: DataFrame = X_train[~X_train.index.isin(novelty.index)].head(
        MIN_SAMPLES
    )
    current_data.loc[:, "isNovelty"], novelty.loc[:, "isNovelty"] = False, True
    sampled_data = current_data.sample(n=int(0.2 * len(current_data)), random_state=42)
    novelty = concat([novelty, sampled_data])

    training_data = current_data.drop(columns=["isNovelty"])
    train_targets = current_data["isNovelty"]
    # only current activity (as novelty)
    testing_data = novelty.drop(columns=["isNovelty"])
    test_targets = novelty["isNovelty"]

    optimizer = BayesianOptimization(
        f=objective_function,
        pbounds={"nu": (0.01, 0.5), "gamma": (1e-4, 1e-1)},
        random_state=42,
    )
    load_logs(optimizer, logs=[LOGS_PATH])
    print("New optimizer is now aware of {} points.".format(len(optimizer.space)))

    optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
    optimizer.maximize(init_points=5, n_iter=25)

    models[activity] = optimizer.max

Activity: 1
New optimizer is now aware of 0 points.

F1 Score (Train): 0.3464052287581699
F1 Score (Test): 0.7996094954269859
Errors in Train 0
Errors in Test 0

F1 Score (Train): 0.4110718492343934
F1 Score (Test): 0.871312553982065
Errors in Train 0
Errors in Test 0

F1 Score (Train): 0.47368421052631576
F1 Score (Test): 0.9369270877133966
Errors in Train 0
Errors in Test 0

F1 Score (Train): 0.3614303959131545
F1 Score (Test): 0.8192044997991161
Errors in Train 0
Errors in Test 0

F1 Score (Train): 0.3932038834951456
F1 Score (Test): 0.8572753871405123
Errors in Train 0
Errors in Test 0

F1 Score (Train): 0.46062567421790723
F1 Score (Test): 0.9461545364803015
Errors in Train 0
Errors in Test 0

F1 Score (Train): 0.49748743718592964
F1 Score (Test): 0.9794092413285942
Errors in Train 0
Errors in Test 0

F1 Score (Train): 0.4802494802494803
F1 Score (Test): 0.9499739447628973
Errors in Train 0
Errors in Test 0

F1 Score (Train): 0.4481236203090508
F1 Score (Test): 0.9358096542280041


: 